In [43]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from random import randint
from fpdf import FPDF
import math

In [44]:
data = pd.read_csv('main_manipulate.csv')
data.shape

(336, 4)

In [45]:
lol = data.values.tolist()
len(lol)

336

In [46]:
def month_to_number(month):
    if month=="January":
        return "1"
    
    elif month=="February":
        return "2"
    
    elif month=="March":
        return "3"
    
    elif month=="April":
        return "4"
    
    elif month=="May":
        return "5"
    
    elif month=="June":
        return "6"
    
    elif month=="July":
        return "7"
    
    elif month=="August":
        return "8"
    
    elif month=="September":
        return "9"
    
    elif month=="October":
        return "10"
    
    elif month=="November":
        return "11"
    
    elif month=="December":
        return "12"
    else:
        return ""
    

In [61]:
import re
import os

import datetime
def create_pdf(info,text_to_pd):
    try:
        name = info[0].replace("’","'")
        name = name.replace("‘","'")
        name = name.replace("”","'")
        name = name.replace("“","'")
        name = "".join(name.split())
        name = name.encode('latin-1', 'replace').decode('latin-1')
        name = name.replace("?","")
        name = name.replace("/","")

        
        year = info[2].split(" ")[2]

        day = info[2].split(" ")[1]
        day = day.replace(",","")

        month_name = info[2].split(" ")[0]
        month = month_to_number(month_name.capitalize())
        
        year = year.replace("?","")
        year = year[:5]
        print(year)
        old_but_after = False
        if int(year)>=2009:
            print("check")
            old_but_after = True
        day = month.replace("?","")
        
        
        day = day.replace("?","")
        if not os.path.exists("pdf/"+year):
            os.makedirs("pdf/"+year)

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font('Arial', '', 14)
        pdf.write(5,"Tittle: "+ info[0])
        pdf.ln()
        pdf.write(5,"Link: " +info[1])
        pdf.ln()
        pdf.write(5,"Date: " +info[2])
        pdf.ln()
        pdf.set_font('Arial', '', 12)
        pdf.ln()
        pdf.ln()
        pdf.ln()
        for line in text_to_pdf:
            text = line.replace("’","'")
            text = text.replace("‘","'")
            text = text.replace("”","'")
            text = text.replace("“","'")
            text = text.encode('latin-1', 'ignore').decode('latin-1')
            pdf.write(5,text)
            pdf.ln()
            
        if old_but_after == True:
            if not os.path.exists("pdf/old_website_after_2009/"+year):
                os.makedirs("pdf/old_website_after_2009/"+year)
            name_file="pdf/old_website_after_2009/"+year+"/" +year +"_" + month+"_"+day+"_"+ name + ".pdf"
            pdf.output(name_file).encode('latin-1','ignore')
        else:    
            #name_file="pdf/"+year+"/" +year +"_" + month+"_"+day+"_"+ name + ".pdf"
            pass
        
    except:
        #print(name)
        pass

In [62]:
def check_china(link):
    try:
        page = requests.get(link).text
        soup = BeautifulSoup(page, 'lxml')
        text = soup.get_text()
        if 'China' in text or 'China' in i[1]:
            return True,text
        else:
            return False,text
    except:
        return False,text
    
    
def remove_news(text):
    if 'News And Press Releases' in text or 'Press Releases for' in text:
        return True
    else:
        return False

In [63]:
def validate_date(date):
    if 'Accessibility' in date or 'Home' in date:
        return math.sqrt(-10)

In [64]:
valid = []
for i in lol:
    if "press-releases" not in i[2] and '.pdf' not in i[2] and 'Press' not in i[2] and 'download' not in i[2] and 'report' not in i[2] and 'assigned' not in i[2]:
        china, text = check_china(i[2])
        news = remove_news(text)
        if china == True and news == False:
            valid.append(i)

In [65]:
from unidecode import unidecode

lista_nova=[]
counter=0
total= 0
cannot_read=[]
not_china=[]
write= []
other=[]
n_article = 0



for i in valid:

    print(i[2])
    if "press-releases" not in i[2] and '.pdf' not in i[2] and 'Press' not in i[2] and 'download' not in i[2] and 'report' not in i[2] and 'assigned' not in i[2]:
        print(n_article)
        info = []
        if counter==50:
            counter=0
            total +=1
            print(total)
            time.sleep(randint(2,15))
        else:
            counter+=1
        link = i[2]
        page = requests.get(link).text
        soup = BeautifulSoup(page, 'lxml')

        content = soup
        info = []
        info_csv=[]
        date = "not_found"
        text = ""


        try:
            print("1 case")
            date = content.find('div',{'id' : 'prdate'}).text
            all_text = content.find('div',{'id' : 'prbody'})
            p_text = content.find_all('p')

        except: 
            try: #colorado website
                print("2 case")
                date = content.find('div',{'class' : 'newsRight'}).text
                p_field = content.find('div',{'class' : 'clear'})
                p_text = p_field.find_all('p')
                date = date.strip()


            except: 
                try: #department of justice

                    print("3 case")
                    date = content.find('div',{'id' : 'contentstart'}).text
                    p_field = content.find('div',{'id' : 'contentstart'})
                    p_text = p_field.find_all('p')
                    date = date.strip()

                except: #ag website
                    try:

                        print("4 case")
                        date_field = list(content.find_all('h2'))
                        date_text=""
                        for line in date_field:
                            date_text+=str(line)
                        date=date_text.split("<br/>")[1]
                        date = date.split("</h2>")[0]
                        p_text = content.find_all('p')

                    except: #california
                        try:

                            print("5 case")
                            #https://www.justice.gov/archive/usao/cac/Pressroom/pr2010/138.html
                            context = content.find('div',{'id' : 'content'})
                            date_field = context.find_all('p')
                            nr = 0
                            for line in data_field:

                                print(line.text)
                                if nr==1:
                                    date = date_field(line).text
                                else:
                                    pass
                                nr +=1
                                print(date)
                            p_text = content.find_all('p')

                        except: #department of justice 2

                            print("6 case")
                            try:
                                data_field = list(content.find('td'))
                                date_text=str(data_field[0])
                                date=date_text.split("<br/>")[1]

                                date=date.split(" ")[1:]
                                date[0] = date[0].capitalize()
                                date = " ".join(date)

                                p_text = content.find_all('p')
                                print("data")
                                print(date)
                            except:
                                try:
                                    print("case 7")
                                    date = content.find('ul').text
                                    check = date.split(" ")[2]
                                    p_text = content.find_all('p')
                                    if "Return to Index of Pre-January 2009 Press Releases" in date:
                                        math.sqrt(-10)
                                except:
                                    try:
                                        print("case8")
                                        date_field = content.find('div').text
                                        date=date_field.split()[3:6]
                                        date[2] = date[2][:4]
                                        date=" ".join(date)
                                        p_text = content.find_all('p')
                                        print(date)
                                    except:
                                        try:
                                            date_field = content.find('center')
                                            date_field = date_field.find_all('b')
                                            n=0
                                            for i in date_field:
                                                n+=1
                                                if n==3:
                                                    date=i.text
                                            p_field = content.find('blockquote')
                                            p_text = p_field.find_all('p')
                                        except:
                                            cannot_read.append(i)






        text_to_pdf=[]

        for p in p_text:

            texto = unidecode(p.text)
            text += texto
            text_to_pdf.append(texto)
        if len(text) < 10:
            pass




        try:


            info.append(unidecode(i[1]))
            info.append(unidecode(i[2]))
            info.append(date)
            #print(info[1])
            #print(info[2])
            year = info[2].split(" ")[2]

            day = info[2].split(" ")[1]
            day = day.replace(",","")

            month_name = info[2].split(" ")[0]
            month = month_to_number(month_name.capitalize())

            info_csv.append(int(year))

            info_csv.append(int(month))

            info_csv.append(int(day))

            info_csv.append(info[0])

            info_csv.append(link)


            info_csv.append(i[3])

            lista_nova.append(info_csv)
            #print(info)
            create_pdf(info,text_to_pdf)
            n_article +=1
        except:
            #print(info[2])
            #print("error")
            #print("year: " +year)
            #print("month: " +month)
            #print("day:" + day)
            write.append(i)


    else:
        other.append(i)

https://www.justice.gov/archive/opa/pr/2007/August/07_nsd_572.html
0
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
AUGUST 2, 2007
2007
https://www.justice.gov/archive/ag/speeches/2006/ag_speech_060622.html
1
1 case
2 case
3 case
4 case
2006
https://www.justice.gov/archive/ag/speeches/2006/ag_speech_0606201.html
2
1 case
2 case
3 case
4 case
2006
https://www.justice.gov/archives/criminal/computer-crime-and-intellectual-property-section
3
1 case
2 case
3 case
4 case
5 case
6 case
case 7
https://www.justice.gov/archive/opa/pr/2007/October/07_crm_836.html
3
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
OCTOBER 22, 2007
2007
https://www.justice.gov/archive/opa/pr/2006/September/06_opa_657.html
4
1 case
2 case
3 case
4 case
5 case
6 case
case 7
https://www.justice.gov/archive/opa/pr/2005/October/05_crm_564.html
4
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
TUESDAY, OCTOBER 25,
https://www.justice.gov/archive/ag/speeches/2008/ag_speech_080328.html
4
1 case
2 case


1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
July 11, 2008
2008
https://www.justice.gov/archive/usao/nys/pressreleases/September12/WuetalCharges.html
33
1 case
2 case
https://www.justice.gov/archive/ag/speeches/1998/may1498.htm
33
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/ag/speeches/1999/july1599.htm
33
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/ag/speeches/2008/ag-speech-080425.html
33
1 case
2 case
3 case
4 case
https://www.justice.gov/archive/ag/speeches/1998/feb0298.htm
33
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/usao/waw/press/categories/drugs.html
33
1 case
2 case
3 case
4 case
5 case

FOR IMMEDIATE RELEASE
THURSDAY, AUGUST 19, 2004
WWW.USDOJ.GOV

not_found
https://www.justice.gov/archive/ag/speeches/1998/dec1098.htm
33
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/usao/ri/news/2011/index.html
33

1 case
2 case
2009
check
https://www.justice.gov/archive/ag/speeches/2005/ag_speech_051110.html
50
1 case
2 case
3 case
4 case
2005
https://www.justice.gov/archive/opa/pr/2008/February/08_crm_150.html
51
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
FEBRUARY 28, 2008
2008
https://www.justice.gov/archive/usao/md/news_2012.html
52
1 case
2 case
3 case
4 case
5 case
https://www.justice.gov/archive/ag/speeches/2000/22400availag.htm
52
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/dag/speeches/1999/dagjune399.htm
52
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/ag/speeches/2007/ag_speech_070301.html
52
1 case
2 case
3 case
4 case
2007
https://www.justice.gov/archive/ag/speeches/1999/july2999.htm
53
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archive/ag/speeches/1999/april0899.htm
53
1 case
2 case
3 case
4 case
5 case
6 case
case 7
case8
https://www.justice.gov/archiv

In [66]:
print("ok")

ok


In [67]:
print(len(cannot_read))
print(len(not_china))
print(len(other))
print(len(write))
print(n_article)

13
0
0
90
54


In [13]:
write

[[6,
  '\nComputer Crime and Intellectual Property Section\n',
  'https://www.justice.gov/archives/criminal/computer-crime-and-intellectual-property-section',
  'intellectual+property+theft+china'],
 [8,
  '\n#06-657: 09-28-06 Bush Administration Releases Report on Intellectual Property Enforcement and Protection\n',
  'https://www.justice.gov/archive/opa/pr/2006/September/06_opa_657.html',
  'intellectual+property+theft+china'],
 [11,
  '\n#05-564: 10-25-05 DEFENDANT FACES 18-COUNT INDICTMENT IN FIRST JOINT CRIMINAL INTELLECTUAL PROPERTY INVESTIGATION BY UNITED STATES AND CHINA\n',
  'https://www.justice.gov/archive/opa/pr/2005/October/05_crm_564.html',
  'intellectual+property+theft+china'],
 [14,
  '\nRemarks Prepared for Delivery by Attorney General Michael B. Mukasey at the Tech Museum of Innovation (2008-03-28)\n',
  'https://www.justice.gov/archive/ag/speeches/2008/ag_speech_080328.html',
  'intellectual+property+theft+china'],
 [2,
  "\nUSDOJ: US Attorney's Office - Eastern Dis

In [15]:
import pdfkit

joinlist = write + cannot_read

failed_pdf=[]
for i in joinlist:
    url = i[2]
    path = "pdf/others/"

    name = i[1].replace("’","'")
    name = name.replace("‘","'")
    name = name.replace("”","'")
    name = name.replace("“","'")
    name = "".join(name.split())
    name = name.encode('latin-1', 'replace').decode('latin-1')
    name = name.replace("?","")
    file_name = name.replace("/","")
    
    
    print("ok")
    try:
        if pdfkit.from_url(url, path + file_name+".pdf"): # Check if method from_url returned True
            #print("Sucessfully created pdf from url")
            pass
        else:
            
            print("Something went wrong")
    except:
        failed_pdf.append(i)

ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                              

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                 

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                   

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                 

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
ok
ok
ok
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                             

In [16]:
len(failed_pdf)

4

In [48]:
df = pd.DataFrame.from_records(lista_nova)

In [49]:
df.to_excel("main_complete.xlsx", header = ["Year", "Month", "Day", "Name", "Link", "Category"]) 

In [17]:
failed = pd.DataFrame.from_records(failed_pdf)
failed.to_excel("failed.xlsx") 